In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
data = pd.read_csv('gemstone.csv')

In [3]:
data_1 = data.drop(columns=['id'])

In [4]:
X = data_1.drop('price',axis=1)
y = data_1['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
X_train.head()

,carat,cut,color,clarity,depth,table,x,y,z
11504,0.41,Ideal,E,VVS2,60.6,56.0,4.85,4.80,2.93
95284,1.23,Very Good,H,VS1,59.9,59.0,6.91,7.01,4.19
184777,1.70,Premium,H,VS2,62.0,58.0,7.61,7.66,4.74
5419,0.33,Ideal,F,VVS1,61.2,56.0,4.47,4.44,2.73
45466,0.33,Very Good,I,SI1,62.1,58.0,4.41,4.45,2.75


In [6]:
oe_cut = OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']])
oe_color = OrdinalEncoder(categories=[['D', 'E', 'F', 'G', 'H', 'I', 'J']])
oe_clarity = OrdinalEncoder(categories=[['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']])


oe_trf = ColumnTransformer([
    ('oe_cut', oe_cut, [0]),
    ('oe_color', oe_color, [1]),
    ('oe_clarity', oe_clarity, [2])
], remainder='passthrough')

scaler_num = StandardScaler()
scaler_cat = StandardScaler()
imputer_numerical = SimpleImputer(strategy='median')
imputer_cat = SimpleImputer(strategy='most_frequent')

pipeline_numerical = Pipeline([
    ('imputer', imputer_numerical),
    ('scaler', scaler_num),
])

pipeline_cat = Pipeline([
    ('imputer', imputer_cat),
    ('oe_trf', oe_trf),
    ('scaler', scaler_cat),
])

preprocessor = ColumnTransformer([
    ('pipeline_numerical', pipeline_numerical, ['carat', 'depth', 'table', 'x', 'y', 'z']),
    ('pipeline_cat', pipeline_cat, ['cut', 'color', 'clarity']),
], remainder='drop')


# X_train_trf = oe_trf.fit_transform(X_train)
# X_test_trf = oe_trf.transform(X_test)

# pd.DataFrame(X_train_trf, columns=oe_trf.get_feature_names_out()).head()

In [7]:
X_train_trf = preprocessor.fit_transform(X_train)
X_train_trf = pd.DataFrame(X_train_trf, columns=preprocessor.get_feature_names_out())
X_train_trf.head()

,pipeline_numerical__carat,pipeline_numerical__depth,pipeline_numerical__table,pipeline_numerical__x,pipeline_numerical__y,pipeline_numerical__z,pipeline_cat__oe_cut__cut,pipeline_cat__oe_color__color,pipeline_cat__oe_clarity__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.874100,-0.936747,1.350746
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-1.137644,0.910853,0.684455
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-0.131772,0.910853,0.018164
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.874100,-0.320880,2.017037
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-1.137644,1.526720,-0.648127


In [8]:
X_test_trf = preprocessor.transform(X_test)
X_test_trf = pd.DataFrame(X_test_trf, columns=preprocessor.get_feature_names_out())
X_test_trf.head()

,pipeline_numerical__carat,pipeline_numerical__depth,pipeline_numerical__table,pipeline_numerical__x,pipeline_numerical__y,pipeline_numerical__z,pipeline_cat__oe_cut__cut,pipeline_cat__oe_color__color,pipeline_cat__oe_clarity__clarity
0,-0.629077,0.258230,-0.120630,-0.600482,-0.581521,-0.572248,0.874100,-1.552614,-0.648127
1,2.605374,-2.148014,-0.120630,2.126042,2.198832,1.959219,-1.137644,0.294987,-1.314417
2,-1.125026,-1.222536,0.921902,-1.374347,-1.414721,-1.469110,-0.131772,-0.936747,2.017037
3,-1.017211,-0.574701,0.921902,-1.158385,-1.161138,-1.194265,-0.131772,1.526720,2.017037
4,0.858771,0.628421,-0.641897,0.947248,0.985258,1.004495,0.874100,0.910853,-0.648127


In [9]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet



def evaluate_model(y_test, model):
    model.fit(X_train_trf, y_train)
    y_pred = model.predict(X_test_trf)
    print('R2 score', r2_score(y_test, y_pred))
    print('MSE', mean_squared_error(y_test, y_pred))
    print('MAE', mean_absolute_error(y_test, y_pred))
    print('\n')
    
evaluate_model(y_test, LinearRegression())
evaluate_model(y_test, Ridge())
evaluate_model(y_test, Lasso())
evaluate_model(y_test, ElasticNet())

R2 score 0.9362906819996049
MSE 1029473.3531156846
MAE 675.0758270067483


R2 score 0.9362900967491632
MSE 1029482.8101268951
MAE 675.1077629781341


R2 score 0.9362869814082755
MSE 1029533.1506505491
MAE 676.2421173665509


R2 score 0.8544967219374031
MSE 2351174.871397875
MAE 1060.9432977143008




In [11]:
score = cross_val_score(LinearRegression(), X_train_trf, y_train, cv=5).mean()
print(score)

0.9367019178783347
